In [8]:
# Mount Google Drive

# PDF TEXT EXTRACT

In [9]:
pip install PyMuPDF

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [12]:
import glob, os

folder = "/content/drive/MyDrive/PSEG_Electric_and_gas_bills_Tchamna_NJ_17_essex"
single_file = "/content/drive/MyDrive/PSEG_Electric_and_gas_bills_Tchamna_NJ_17_essex/2021-11-16-PSEG_Gas_Electric_bill_Tchamna_essex.pdf"

absolute_file_name_list = glob.glob(folder+"/*.pdf")

relative_file_name_list = [os.path.basename(i) for i in absolute_file_name_list]
relative_file_name_list[0]


'2023-01-24-PSEG_Gas_Electric_bill_Tchamna_essex.pdf'

In [13]:
import fitz

import re, os
import pandas as pd

def replace(s,dict_={"Plus Gas supply charges - SmartEnergy Holdings, LLC":"SmartEnergy_Gas_supply_charges","supply from PSE&G, your cost would be":"supply from PSE&G cost would be"}):
    for i in dict_:
      s = s.replace(i,dict_[i])
    return s 
    
df = pd.DataFrame()

for k in range(len(absolute_file_name_list)):
    
  # doc = fitz.open(single_file)
  path_to_file = absolute_file_name_list[k]
  doc = fitz.open(path_to_file)
  date = os.path.basename(path_to_file)[:10]

  doc_list = []
  for page in doc:
    text = page.get_text()
    text = text.replace("�","'")
    doc_list.append(text)

  # s = "Gas supply charges - SmartEnergy Holdings, LLC $184.29 teys and SmartEnergy $455.5"

  # pat_SmartEnergy = "(SmartEnergy[\w+,\s]*)\s+\$(\d+\.\d+)+"
  pat_SmartEnergy = "([\w+,\*\/_\'&\s-]*)\s+\$(\d+\.\d+)+"


  for i in range(len(doc_list)):
    globals()[f"fees_page_{i+1}"] = re.findall(f"{pat_SmartEnergy}", doc_list[i],flags=re.IGNORECASE)  
    globals()[f"fees_page_{i+1}"] = [elt+(date,i+1) for elt in globals()[f"fees_page_{i+1}"]]

    df = df.append(globals()[f"fees_page_{i+1}"])
  # pd.DataFrame(fees_page_2)
  # df.columns = ["Charge_Name","Amount($)","Date","Page"]
  
  # display(df)

  # df["Charge_Name"] = df["Charge_Name"].apply(lambda x: x.split("\n")[-1])
  # df["Amount($)"] = df["Amount($)"].astype("float")
  df[0] = df[0].apply(lambda x: x.split("\n")[-1])
  df[1] = df[1].astype("float")

  # df.sort_values(by="Amount($)")
  # 0.045366	* 328 = 14.880047999999999
  keep = ["Balance remaining from your last bill","Total tax amount","Energy 328 kWh at","1 Water Heater*",
  "Monthly service charge","WorryFree Protection Plan charge","Monthly service charge", "1 Gas Furnace/Boiler*",
  "Current Amount","Total electric delivery charges", "be","Total SmartEnergy Charges","Total gas charges",
  "Total gas delivery charges","supply from PSE&G, your cost would be","Plus Gas supply charges - SmartEnergy Holdings, LLC","This month's charges and credits"] 

  df = df[df[0].isin(keep)]

  # df.query(f"Charge_Name in {keep}")
  # df.query(f"Charge_Name in @keep")
  

  df = df.drop_duplicates(subset=df.columns[:-1])
  # df[0] = df[0].apply(replace)

df.columns = ["Charge_Name","Amount($)","Date","Page"]

df.to_csv("PSEG_Gas_Electricity_Charge_Tchamna.csv")
df

,Charge_Name,Amount($),Date,Page
0,Balance remaining from your last bill,0.00,2023-01-24,1
1,This month's charges and credits,463.02,2023-01-24,1
5,Plus Gas supply charges - SmartEnergy Holdings...,184.29,2023-01-24,2
0,"supply from PSE&G, your cost would be",162.32,2023-01-24,3
1,Monthly service charge,8.62,2023-01-24,3
...,...,...,...,...
1,Monthly service charge,3.74,2021-11-16,3
6,Total gas delivery charges,42.90,2021-11-16,3
10,Total gas charges,68.91,2021-11-16,3
1,Monthly service charge,2.14,2021-11-16,4
